# Parameters

In [ ]:
table_name <- "demux_BH3KTLDMXY"
experiment_name <- "gm12878_fresh_ATAC"
papermill <- FALSE
# frag.path <- NA
h5.path <- NA

In [ ]:
papermill <- as.logical(papermill)

# Install Libraries

Install information can be found [here](https://github.com/aertslab/cisTopic)\
Vignette [here](http://htmlpreview.github.io/?https://github.com/aertslab/cisTopic/blob/master/vignettes/WarpLDA_10X_workflow.html)

In [ ]:
# Libraries for installing libraries
if (!requireNamespace("BiocManager", quietly = TRUE))
{
    message("Installing BiocManager")
    install.packages("BiocManager", repos = "https://cran.r-project.org")
}
if (!requireNamespace("AnVIL", quietly = TRUE))
{
    message("Installing AnVIL")
    BiocManager::install("AnVIL")
}
if (!requireNamespace("devtools", quietly = TRUE))
{
    message("Installing devtools")
    install.packages("devtools")
}

# Cistopic Libraries
if (!requireNamespace("GenomeInfoDbData", quietly = TRUE))
    AnVIL::install("GenomeInfoDbData")
if (!requireNamespace("GenomicRanges", quietly = TRUE))
    AnVIL::install("GenomicRanges")
if (!requireNamespace("Rsamtools", quietly = TRUE))
    AnVIL::install("Rsamtools")
# if (!requireNamespace("GenomicAlignments", quietly = TRUE))
#     AnVIL::install("GenomicAlignments")
# if (!requireNamespace("rtracklayer", quietly = TRUE))
#     AnVIL::install("rtracklayer")
# if (!requireNamespace("SumarizedExperiment", quietly = TRUE))
#     AnVIL::install("SumarizedExperiment")

if (!requireNamespace("cisTopic", quietly = TRUE))
{
    message("Installing cisTopic")
    Sys.setenv(LIBARROW_MINIMAL = "false")
    devtools::install_github("aertslab/cisTopic")
}
if (!requireNamespace("Seurat", quietly = TRUE))
{
    message("Installing Seurat")
#     devtools::reload(pkg = pkgload::inst("pillar"), quiet = FALSE)
    AnVIL::install("Seurat")
}
if (!requireNamespace("densityClust", quietly = TRUE))
{
    message("Installing densityClust")
    install.packages("densityClust")
}
if (!requireNamespace("scatterplot3d", quietly = TRUE))
{
    message("Installing scatterplot3d")
    install.packages("scatterplot3d")
}
if (!requireNamespace("umap", quietly = TRUE))
{
    message("Installing umap")
    install.packages("umap")
}
if (!requireNamespace("hdf5r", quietly = TRUE))
{
    message("Installing hdf5r")
    install.packages('hdf5r')
}

In [ ]:
suppressMessages(library(AnVIL))
suppressMessages(library(cisTopic))
suppressMessages(library(densityClust))
suppressMessages(library(Rtsne))
suppressMessages(library(Seurat))
set.seed(1234)

# Load Workspace Tables

In [ ]:
if (!papermill){
    table <- avtable(table_name)
}

In [ ]:
get_file <- function(path){
    dest <- getwd()
    gsutil_cp(path, dest)
    name <- basename(path)
    return(name)
}

# if (!papermill){
#     frag.path <- get_file(table$atac_fragment_file_raw[table[, sprintf('%s_id', table_name)] == experiment_name])
# }
# barcode_path <- get_file(table$atac_fragment_file_raw[table$demux_BH3KTLDMXY_id == 'gm12878_fresh_ATAC'])

In [ ]:
if (!papermill){
    wsData <- avdata()
    h5.path <- get_file(wsData$value[wsData$key=='cell_peak_mat'])
}

# Generate UMAP

In [ ]:
counts <- Read10X_h5(h5.path)

In [ ]:
cisTopicObject <- createcisTopicObject(counts, project.name='atac')

In [ ]:
cisTopicObject <- runWarpLDAModels(cisTopicObject, topic=c(50), seed=987, nCores=4, iterations = 500, addModels=FALSE)

In [ ]:
cisTopicObject <- selectModel(cisTopicObject, type='maximum')


In [ ]:
cisTopicObject <- runtSNE(cisTopicObject, target='cell', seed=123, pca=FALSE, method='Probability')

cisTopicObject <- runUmap(cisTopicObject, target='cell', seed=123, method='Probability')

In [ ]:
cellassign <- modelMatSelection(cisTopicObject, 'cell', 'Probability')

In [ ]:
set.seed(123)
DR <- Rtsne(t(cellassign), pca=F)
DRdist <- dist(DR$Y)
dclust <- densityClust(DRdist,gaussian=T)

In [ ]:
dir.create("plots", showWarnings=F)
printPNG <- function(filename, plotObject, papermill, wf=1, hf=1){
	options(repr.plot.width = 7*wf, repr.plot.height = 7*hf)
	print(plotObject)
	if(papermill){
		png(sprintf("plots/%s.png", filename), width=480*wf, height=480*hf)
		print(plotObject)
		dev.off()
	}
}

dclust <- findClusters(dclust, rho = 40, delta = 2.5)

In [ ]:
# Check thresholds
options(repr.plot.width=6, repr.plot.height=6)
plot(dclust$rho,dclust$delta,pch=20,cex=0.6,xlab='rho', ylab='delta')
points(dclust$rho[dclust$peaks],dclust$delta[dclust$peaks],col="red",pch=20,cex=0.8)
text(dclust$rho[dclust$peaks]-2,dclust$delta[dclust$peaks]+1.5,labels=dclust$clusters[dclust$peaks])
abline(v=50)
abline(h=2.5)

In [ ]:
# Add cluster information
densityClust <- dclust$clusters
densityClust <- as.data.frame(densityClust)
rownames(densityClust) <- cisTopicObject@cell.names
colnames(densityClust) <- 'densityClust'
densityClust[,1] <- as.factor(densityClust[,1])
cisTopicObject <- addCellMetadata(cisTopicObject, densityClust)

In [ ]:
par(mfrow=c(2,2))
plotFeatures(cisTopicObject, method='Umap', target='cell', topic_contr=NULL, 
             colorBy=c('nCounts', 'nAcc','densityClust'), 
             cex.legend = 0.8, factor.max=.75, dim=2, legend=TRUE, col.low='darkgreen', 
             col.mid='yellow', col.high='brown1', intervals=10)